# Problem 3: Vehicle Routing Problem
## Alexander Belov

* $c$ = couriers, $k$ = clients
* $s_c$ = salary of c-th courier
* $p_c$ = path of c-th courier; $p_{c,i}$ = i-th stop of c-th courier
* $d$ = distance matrix
* $l_{c,i}$ = amount of load (+) or unload (-) of c-th courier on his i-th stop
* $D_k$ = demand of k-th client
* $C_c$ = capacity of c-th driver

$$
\sum_{c} s_c \left( \sum_{i=0}^{len(p_{c})-2} d(p_{c,i}; p_{c,i+1}) \right) \rightarrow \min_{l, p}
$$

Ограничения задачи:
* В каджый момент времени загрузка курьера не превышает вместимости
$$
\forall c \, \forall t < len(p_c) \quad \sum_{i \leq t} l_{c,i} \leq C_{c}
$$

* Загружать можно только на складе
$$
l_{c,i} > 0 \implies p_{c,i} = 0
$$

* Все заказы выполнены
$$
\forall k \quad\sum_{\substack{c \\ i: p_{c,i} = k}} -l_{c,i} = D_{k}
$$

* Клиенты не бывают перегружены

Ограничения моего решения (я рассматриваю только такие решения):

* Каждая остановка посещается только один раз одним курьером (кроме 0й, это магазин)
$$
\forall k \neq 0 \quad \exists ! c, i: \quad p_{c,i} = k
$$
* Курьер сразу выгружает весь заказ
$$
\forall k \forall c \forall i: \quad p_{c,i} = k \quad \implies \quad l_{c,i} = D_{k}
$$
* Курьеры не могут вернуться в магазин для дозагрузки; начинаем и заканчиваем в магазине
$$
p_{c,0} = p_{c,-1} = 0
$$

Алгоритм решения:
1. Сгенерировать первое поколение: заказы распределяются по курьерам, затем для каждого курьера решается TSP по его заказам.
2. Сгенерировать новое поколение: мутировать каждое решение из пред. поколения и отобрать лучших. Мутация решения - это  перекидыание заказов между водителями.

In [1]:
import numpy as np
from dataclasses import dataclass
from typing import List

Исходные данные.

In [2]:
@dataclass
class ProblemInstance:
    distance_matrix: np.ndarray = np.asarray([
      [0, 5.48, 7.76, 6.96, 5.82, 2.74, 5.02, 1.94, 3.08, 1.94, 5.36, 5.02, 3.88, 3.54, 4.68, 7.76, 6.62],
      [5.48, 0, 6.84, 3.08, 1.94, 5.02, 7.30, 3.54, 6.96, 7.42, 10.84, 5.94, 4.80, 6.74, 10.16, 8.68, 12.10],
      [7.76, 6.84, 0, 9.92, 8.78, 5.02, 2.74, 8.10, 4.68, 7.42, 4.00, 12.78, 11.64, 11.30, 7.88, 15.52, 7.54],
      [6.96, 3.08, 9.92, 0, 1.14, 6.50, 8.78, 5.02, 8.44, 8.90, 12.32, 5.14, 6.28, 8.22, 11.64, 5.60, 13.58],
      [5.82, 1.94, 8.78, 1.14, 0, 5.36, 7.64, 3.88, 7.30, 7.76, 11.18, 4.00, 5.14, 7.08, 10.50, 6.74, 12.44],
      [2.74, 5.02, 5.02, 6.50, 5.36, 0, 2.28, 3.08, 1.94, 2.40, 5.82, 7.76, 6.62, 6.28, 5.14, 10.50, 7.08],
      [5.02, 7.30, 2.74, 8.78, 7.64, 2.28, 0, 5.36, 1.94, 4.68, 3.54, 10.04, 8.90, 8.56, 5.14, 12.78, 4.80],
      [1.94, 3.54, 8.10, 5.02, 3.88, 3.08, 5.36, 0, 3.42, 3.88, 7.30, 4.68, 3.54, 3.20, 6.62, 7.42, 8.56],
      [3.08, 6.96, 4.68, 8.44, 7.30, 1.94, 1.94, 3.42, 0, 2.74, 3.88, 8.10, 6.96, 6.62, 3.20, 10.84, 5.14],
      [1.94, 7.42, 7.42, 8.90, 7.76, 2.40, 4.68, 3.88, 2.74, 0, 3.42, 5.36, 4.22, 3.88, 2.74, 8.10, 4.68],
      [5.36, 10.84, 4.00, 12.32, 11.18, 5.82, 3.54, 7.30, 3.88, 3.42, 0, 8.78, 7.64, 7.30, 3.88, 11.52, 3.54],
      [5.02, 5.94, 12.78, 5.14, 4.00, 7.76, 10.04, 4.68, 8.10, 5.36, 8.78, 0, 1.14, 3.08, 6.50, 2.74, 8.44],
      [3.88, 4.80, 11.64, 6.28, 5.14, 6.62, 8.90, 3.54, 6.96, 4.22, 7.64, 1.14, 0, 1.94, 5.36, 3.88, 7.30],
      [3.54, 6.74, 11.30, 8.22, 7.08, 6.28, 8.56, 3.20, 6.62, 3.88, 7.30, 3.08, 1.94, 0, 3.42, 4.22, 5.36],
      [4.68, 10.16, 7.88, 11.64, 10.50, 5.14, 5.14, 6.62, 3.20, 2.74, 3.88, 6.50, 5.36, 3.42, 0, 7.64, 1.94],
      [7.76, 8.68, 15.52, 5.60, 6.74, 10.50, 12.78, 7.42, 10.84, 8.10, 11.52, 2.74, 3.88, 4.22, 7.64, 0, 7.98],
      [6.62, 12.10, 7.54, 13.58, 12.44, 7.08, 4.80, 8.56, 5.14, 4.68, 3.54, 8.44, 7.30, 5.36, 1.94, 7.98, 0]
    ])
    flower_market_id: int = 0
    demands: np.ndarray = np.asarray([0, 100, 100, 200, 400, 200, 400, 800, 800, 100, 200, 100, 200, 400, 400, 800, 800])
    num_couriers: int = 4
    salary_per_km: np.ndarray = np.asarray([100] * 4)
    courier_max_load: np.ndarray = np.asarray([2500, 2000, 1000, 500])

Класс решения и подсчет стоимости решения.

In [3]:
@dataclass
class Solution:
    paths: List[np.ndarray]
    loads: List[np.ndarray]


def cost(sol: Solution, pr: ProblemInstance):
    '''
    Считает стоимость решения
    '''
    total_cost = 0
    sat_demands = np.zeros(len(pr.demands), dtype=int)
    assert len(sol.paths) == len(sol.loads)
    n_cars = len(sol.paths)
    for d in range(n_cars):

        assert sol.paths[d][0] == 0
        assert sol.paths[d][-1] == 0
        assert sol.paths[d].size == sol.loads[d].size

        cur_car_load = 0
        for i in range(sol.paths[d].size - 1):
            cur_stop, next_stop = sol.paths[d][i], sol.paths[d][i+1]
            total_cost += pr.salary_per_km[d] * pr.distance_matrix[cur_stop, next_stop]
            cur_car_load += sol.loads[d][i]
            sat_demands[cur_stop] -= sol.loads[d][i]

            assert (cur_stop == 0) or (sol.loads[d][i] <= 0), (d,i) # можно забирать цветы только на базе
            assert 0 <= cur_car_load <= pr.courier_max_load[d]
            assert (cur_stop == 0) or (0 <= sat_demands[cur_stop] <= pr.demands[cur_stop])

    sat_demands[0] = 0
    assert np.all(sat_demands == pr.demands), (pr.demands - sat_demands)

    return total_cost

Распределение заказов по курьерам.

In [4]:
def check_assignment(order2car, orders, capacities):
    '''
    Проверяет, что каждый заказ назначен в корректную машину,
    и что вместимости машин не превышены
    '''
    n_cars = len(capacities)
    assert np.all((0 <= order2car) & (order2car < n_cars))
    loads = np.zeros(n_cars, dtype=int)
    for car in range(n_cars):
        loads[car] = orders[order2car == car].sum()
    return np.all(loads <= capacities)


def assign_best_fit(orders, cars):
    '''
    Эвристика раскладывания заказов по курьерам
    Пробуем класть самый большой заказ в самую большую машину, потом в меньшую и т. д. 
    '''
    # TODO: сделать очередность машин не по изначальной
    # емкости а по текущему свободному месту ?
    nth_largest_order = np.argsort(-orders)
    nth_largest_car = np.argsort(-cars)

    order2car = np.empty_like(orders, dtype=int)
    space = cars.copy()
    for i in range(len(orders)):
        fit = False
        for j in range(len(cars)):
            if space[nth_largest_car[j]] >= orders[nth_largest_order[i]]:
                order2car[nth_largest_order[i]] = nth_largest_car[j]
                space[nth_largest_car[j]] -= orders[nth_largest_order[i]]
                fit = True
                break
        if not fit:
            raise ValueError('failed to fit')

    return order2car


def mutate_assignment(order2car, n_cars, orders):
    '''
    Мутация распределения заказов по курьерам.
    '''
    while True:
        # случайно выбрать двух курьеров
        car1, car2 = np.random.choice(n_cars, replace=False, size=2)
        car1_orders = np.nonzero(order2car == car1)[0]
        car2_orfers = np.nonzero(order2car == car2)[0]

        # выбрать 1 или 2 заказа у первого
        mask1 = np.zeros_like(car1_orders, dtype=bool)
        n_mut = np.random.choice([1, 2])
        i1 = np.random.choice(car1_orders.size, size=n_mut)
        mask1[i1] = True

        # выбрать 1 или 2 заказа у второго
        mask2 = np.zeros_like(car2_orfers, dtype=bool)
        n_mut = np.random.choice([1, 2])
        i2 = np.random.choice(car2_orfers.size, size=n_mut)
        mask2[i2] = True

        # если суммарный вес заказов одинаковый, то меняемся заказами
        if orders[car1_orders[mask1]].sum() == orders[car2_orfers[mask2]].sum():
            order2car_new = order2car.copy()
            order2car_new[car1_orders[mask1]] = car2
            order2car_new[car2_orfers[mask2]] = car1
            return order2car_new

TSP.

In [5]:
def tsp_mutation(sol):
    '''
    Мутация маршрута: меняет порядок двух соседних остановок.
    '''
    mut = sol.copy()
    i = np.random.randint(low=0, high=sol.size-1)
    mut[[i, i+1]] = sol[[i+1, i]]
    return mut


def path_len(dists, path):
    l = 0
    for i in range(len(path)-1):
        l += dists[path[i], path[i+1]]
    l += dists[path[0], path[1]] + dists[path[-1], 0]
    return l


def solve_tsp(dists, pop_size, n_gens):
    '''
    Решить TSP с помощью генетического алгоритма.
    Возвращает массив вида [0, 1, 2, 3, 0],
    что соответствует маршруту 0 -> 1 -> 2 -> 3 -> 0.
    '''

    pop = []
    for i in range(pop_size):
        sol = np.random.permutation(np.arange(1, dists.shape[0]))
        sol_cost = path_len(dists, sol)
        pop.append((sol, sol_cost))

    pop.sort(key=lambda x: x[1])

    for it in range(n_gens):

        children = []
        for i in range(pop_size):
            sol, _ = pop[i]
            ch = tsp_mutation(sol)
            ch_cost = path_len(dists, ch)
            children.append((ch, ch_cost))

        pop += children
        pop.sort(key=lambda x: x[1])
        del pop[pop_size:]
    
    sol, sol_cost = pop[0]
    return np.hstack([[0], sol, [0]])

Эта часть кода берет распределение заказов и решает TSP для каждого курьера, затем собирает все воедино в  решение.

In [6]:
def order2car_from_sol(sol: Solution, n_stops_nincl_shop):
    '''
    По решению sol вернет массив: какому заказу какой курьер назначен.
    '''
    order2car = np.empty(shape=n_stops_nincl_shop, dtype=int)
    n_cars = len(sol.paths)
    for i_car in range(n_cars):
        for i_stop in range(1, len(sol.paths[i_car]) - 1):
            order2car[sol.paths[i_car][i_stop]-1] = i_car
    return order2car


def sol_from_order2car_with_tsp_paths(
    order2car,
    tsp_pop_size,
    tsp_n_gens,
    pr: ProblemInstance
):
    '''
    По клиентам, раскиданным по курьерам согласно order2car,
    вернет решение, где каждому водителю назначен кратчайший TSP маршрут
    по его клиентам.
    '''
    n_cars = pr.salary_per_km.size
    sol = Solution(paths=[], loads=[])
    for c in range(n_cars):  
        # остановки этого водителя, не считая остановку №0 (магазин)
        stops_not_incl_shop = np.nonzero(order2car == c)[0] + 1
        # остановки этого водителя, считая остановку №0 (магазин)
        stops_incl_shop = np.hstack([[0], stops_not_incl_shop])
        # подматрица расстояний только для остановок этого водителя
        dists_submat = pr.distance_matrix[np.ix_(stops_incl_shop, stops_incl_shop)]
        # решаем TSP для водителя
        tsp_sol = solve_tsp(
            dists=dists_submat * pr.salary_per_km[c],
            pop_size=tsp_pop_size,
            n_gens=tsp_n_gens,
        )
        path = stops_incl_shop[tsp_sol]
        sol.paths.append(path)

        # у каждого клиента выгружаем целиком его заказ.
        delta_load = np.zeros_like(path, dtype=int)
        for i in range(1, len(path) - 1):
            delta_load[i] = -pr.demands[path[i]]
            delta_load[0] += pr.demands[path[i]]
        sol.loads.append(delta_load)

    return sol

Мутация решения.

In [7]:
def mutate_sol(
    sol: Solution,
    pr: ProblemInstance,
    tsp_n_gens: int,
    tsp_pop_size: int,
):
    '''
    Мутация решения: перекидать клиентов от одного курьера к другому,
    затем назначить каждому курьеру кратчайший TSP маршрут по его клиентам.
    '''
    # вытаскиваем распределение заказов по курьерам
    order2car = order2car_from_sol(sol=sol, n_stops_nincl_shop=pr.demands.size-1)
    # мутируем распределение заказов по курьерам
    order2car = mutate_assignment(
        order2car=order2car,
        n_cars=pr.courier_max_load.size,
        orders=pr.demands[1:]
    )
    assert check_assignment(order2car, pr.demands[1:], pr.courier_max_load)
    # по распределениям строим tsp маршруты для каждого курьера по его заказам
    return sol_from_order2car_with_tsp_paths(
        order2car=order2car,
        tsp_pop_size=tsp_pop_size,
        tsp_n_gens=tsp_n_gens,
        pr=pr
    )

Основная часть

In [8]:
tsp_n_gens = 10
tsp_pop_size = 10
pop_size = 50
n_gens = 40
np.random.seed(0)
pr = ProblemInstance()

print('Generating initial population...')
pop = []
for i in range(pop_size):
    order2car = assign_best_fit(pr.demands[1:], pr.courier_max_load)
    sol = sol_from_order2car_with_tsp_paths(order2car, tsp_pop_size, tsp_n_gens, pr)
    sol_cost = cost(sol=sol, pr=pr)
    pop.append((sol, sol_cost))

pop.sort(key=lambda x: x[1])

for it in range(n_gens):
    print('iteration', it)

    children = []
    for i in range(pop_size):
        sol, _ = pop[i]
        ch = mutate_sol(sol, pr, tsp_n_gens, tsp_pop_size)
        ch_cost = cost(sol=ch, pr=pr)
        children.append((ch, ch_cost))

    pop += children
    pop.sort(key=lambda x: x[1])
    del pop[pop_size:]

    print(*pop[0], '', sep='\n')

Generating initial population...
iteration 0
Solution(paths=[array([ 0, 13, 15,  4,  1,  7,  0]), array([ 0, 16,  6,  8,  0]), array([ 0, 10, 14,  5,  3,  0]), array([ 0, 11, 12,  9,  2,  0])], loads=[array([2500, -400, -800, -400, -100, -800,    0]), array([2000, -800, -400, -800,    0]), array([1000, -200, -400, -200, -200,    0]), array([ 500, -100, -200, -100, -100,    0])])
9176.0

iteration 1
Solution(paths=[array([ 0, 13, 15,  4,  1,  7,  0]), array([ 0, 16,  6,  8,  0]), array([ 0,  3,  2, 10, 14,  9,  0]), array([ 0, 11, 12,  5,  0])], loads=[array([2500, -400, -800, -400, -100, -800,    0]), array([2000, -800, -400, -800,    0]), array([1000, -200, -100, -200, -400, -100,    0]), array([ 500, -100, -200, -200,    0])])
8332.0

iteration 2
Solution(paths=[array([ 0, 13, 15,  4,  1,  7,  0]), array([ 0, 16,  6,  8,  0]), array([ 0,  3,  2, 10, 14,  9,  0]), array([ 0, 11, 12,  5,  0])], loads=[array([2500, -400, -800, -400, -100, -800,    0]), array([2000, -800, -400, -800,    